# Giriş İşlemleri

In [ ]:
%%capture
!pip install kagglehub pandas datasets scikit-learn torch transformers[torch] matplotlib seaborn

In [ ]:
import kagglehub
import pandas as pd
import os
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import torch
import json

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.
    
    Args:
        None
        
    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "spam_message_classifier_tr" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")
model_cikti_dizini = os.path.join(kok_dizin, "kilometre_tasi")
model_log_dizini = os.path.join(kok_dizin, "log")
eval_sonuc_json_path = os.path.join(model_log_dizini, "eval_sonuc.json")
train_sonuc_json_path = os.path.join(model_log_dizini, "train_sonuc.json")
if not os.path.exists(model_cikti_dizini):
    os.makedirs(model_cikti_dizini)
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

In [ ]:
cihaz = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {cihaz}")

# Veri Kümesi Okuma

## Veri kümesi okuma fonksiyonu

In [ ]:
def veri_kumesi_oku(veri_kumesi_adi: str) -> pd.DataFrame:
    """
    Kaggle veri kümesini indirir ve pandas DataFrame olarak döndürür.

    Bu fonksiyon kagglehub API'sini kullanarak belirtilen veri kümesini indirir,
    "TurkishSMSCollection.csv" dosyasını yükler ve bir pandas DataFrame olarak geri döndürür.
    CSV dosyası noktalı virgül (;) ile ayrılmış ve UTF-8 kodlamasıyla okunur.

    Args:
        veri_kumesi_adi (str): İndirilecek Kaggle veri kümesinin yolu ve adı
                               (örn: "username/dataset-name")

    Returns:
        pd.DataFrame: İndirilen ve işlenen veri kümesini içeren pandas DataFrame
    """
    # Kaggle veri kümesini indir ve geçici dosya yolunu al
    gecici_yol = os.path.join(kagglehub.dataset_download(veri_kumesi_adi), "TurkishSMSCollection.csv")
    
    # CSV dosyasını pandas DataFrame olarak oku ve döndür
    return pd.read_csv(gecici_yol, sep=";", encoding="utf-8")

## Veri kümesini oku

In [ ]:
# Veri kümesi bilgileri
dataset_name = "onurkarasoy/turkish-sms-collection"

veri_kumesi = veri_kumesi_oku(dataset_name)
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesinin gereksiz kolonlarını silme

In [ ]:
tohum_degeri = 571
# gereksiz sütunları kaldır
veri_kumesi = veri_kumesi[['Message', 'Group']]
# veri kümesini karıştır
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini karıştırma

In [ ]:
karismis_veri_kumesi = veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
karismis_veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini eğitim ve sınama olarak parçalama

In [ ]:
def veri_kumesini_oranli_bol(veri_kumesi: pd.DataFrame, test_orani: float = 0.1, 
                            dogrulama_orani: float = 0.1, tohum_degeri: int = 42) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Veri kümesini, Group kolonundaki değerlerin oranlarını koruyarak üçe böler.
    
    Bu fonksiyon, verilen veri kümesini her bir grubun oransal dağılımını koruyarak
    eğitim, doğrulama ve test setlerine ayırır. Bu sayede her bir veri setinde
    grup dağılımları orijinal veri kümesindeki gibi korunmuş olur.
    
    Args:
        veri_kumesi (pd.DataFrame): Bölünecek veri kümesi
        test_orani (float): Test veri kümesinin oranı (varsayılan: 0.1)
        dogrulama_orani (float): Doğrulama veri kümesinin oranı (varsayılan: 0.1)
        tohum_degeri (int): Random state değeri (tekrarlanabilirlik için)
    
    Returns:
        tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: (egitim_veri_kumesi, dogrulama_veri_kumesi, test_veri_kumesi) - 
                                                         Oransal olarak bölünmüş veri kümeleri
    """
    # Benzersiz grup değerlerini bul
    gruplar = veri_kumesi['Group'].unique()
    
    # Her bölüm için boş listeler oluştur
    egitim_liste = []
    dogrulama_liste = []
    test_liste = []
    
    # Her grup için ayrı ayrı bölme işlemi yap
    for grup in gruplar:
        # Sadece bu gruba ait verileri seç
        grup_verisi = veri_kumesi[veri_kumesi['Group'] == grup]
        
        # Bu grubu karıştır - tekrarlanabilirlik için tohum değeri kullanılır
        karisik_grup = grup_verisi.sample(frac=1, random_state=tohum_degeri)
        
        # Bölme noktalarını hesapla
        test_bolme_noktasi = int(len(karisik_grup) * test_orani)
        dogrulama_bolme_noktasi = int(len(karisik_grup) * dogrulama_orani)
        
        # Bu gruptan test, doğrulama ve eğitim verilerini ayır
        # İlk kısım test verisi için ayrılır
        test_grubu = karisik_grup[:test_bolme_noktasi]
        # Test verisinden sonraki kısım doğrulama verisi için ayrılır
        dogrulama_grubu = karisik_grup[test_bolme_noktasi:test_bolme_noktasi+dogrulama_bolme_noktasi]
        # Kalan kısım eğitim verisi olur
        egitim_grubu = karisik_grup[test_bolme_noktasi+dogrulama_bolme_noktasi:]
        
        # İlgili listelere ekle
        test_liste.append(test_grubu)
        dogrulama_liste.append(dogrulama_grubu)
        egitim_liste.append(egitim_grubu)
    
    # Her bir grup için ayrılan verileri birleştir
    sinama_veri_kumesi = pd.concat(test_liste).reset_index(drop=True)
    dogrulama_veri_kumesi = pd.concat(dogrulama_liste).reset_index(drop=True)
    egitim_veri_kumesi = pd.concat(egitim_liste).reset_index(drop=True)
    
    # Son bir karıştırma işlemi yaparak veri sırasını rasgele hale getir
    sinama_veri_kumesi = sinama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    dogrulama_veri_kumesi = dogrulama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    egitim_veri_kumesi = egitim_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    
    # Sonuçları döndür
    return egitim_veri_kumesi, dogrulama_veri_kumesi, sinama_veri_kumesi

In [ ]:
egitim_veri, dogrulama_veri, sinama_veri = veri_kumesini_oranli_bol(karismis_veri_kumesi, tohum_degeri=tohum_degeri)
egitim_veri.head(5)  # İlk 5 satırı göster
dogrulama_veri.head(5)
sinama_veri.head(5)  # İlk 5 satırı göster

## Veri kümelerini eğitime hazır hale getirme fonksiyonu

In [ ]:
def verileri_hazirla(df: pd.DataFrame, tokenizer: Any) -> Dataset:
    """
    Veri çerçevesini dönüştürüp model için kullanılacak Hugging Face Dataset formatına çevirir.
    
    Bu fonksiyon metin sınıflandırma modeli için gerekli hazırlıkları yapar:
    1. Etiketleri BERT formatına (0 ve 1) dönüştürür
    2. Metinleri ve etiketleri kullanarak Dataset nesnesi oluşturur
    3. Metinleri tokenize eder ve model girdisi için hazırlar
    
    Args:
        df (pd.DataFrame): İşlenecek veri çerçevesi, "Message" ve "Group" sütunları içermelidir
        tokenizer (Any): Metinleri tokenize etmek için kullanılacak tokenizer (genellikle BertTokenizer)
        
    Returns:
        Dataset: Tokenize edilmiş ve model için hazırlanmış veri kümesi
    """
    # Group değerlerini 0 ve 1'e dönüştür (Group=1 -> 0, Group=2 -> 1)
    # BERT sınıflandırma için 0-based indeksler kullanır
    labels = df["Group"].apply(lambda x: 0 if x == 1 else 1).tolist()
    
    # DataFrame'den Hugging Face Dataset formatına dönüştür
    dataset = Dataset.from_dict({
        "text": df["Message"].tolist(),  # Metin verileri
        "label": labels                 # Dönüştürülmüş etiketler
    })
    
    # Metinleri tokenize etmek için iç fonksiyon tanımla
    def tokenize_function(examples: dict) -> dict:
        """
        Örnek verilerini tokenize eder.
        
        Args:
            examples (dict): İşlenecek örneklerin sözlüğü
            
        Returns:
            dict: Tokenize edilmiş örnekler
        """
        return tokenizer(
            examples["text"],
            padding="max_length",   # Tüm örnekleri aynı uzunluğa getir
            truncation=True,        # Gerekirse metni kırp
            max_length=512,         # BERT için maksimum uzunluk genellikle 512'dir
        )
    
    # Dataset'i tokenize et (batched=True ile daha hızlı çalışır)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    
    return tokenized_dataset

## Sınama sonucu hesaplama fonksiyonu

In [ ]:
def compute_metrics(eval_pred: tuple) -> dict:
    """
    Model tahminleri için değerlendirme ölçütlerini hesaplar.
    
    Bu fonksiyon, model değerlendirmesi sırasında tahminleri ve gerçek etiketleri alarak
    doğruluk (accuracy), F1 skoru, kesinlik (precision) ve duyarlılık (recall) değerlerini hesaplar.
    İkili sınıflandırma problemi için optimize edilmiştir.
    
    Args:
        eval_pred (tuple): İki elemanlı bir tuple:
            - predictions: Model tarafından yapılan tahminlerin olasılık değerleri (logits)
            - labels: Gerçek etiket değerleri
            
    Returns:
        dict: Hesaplanan değerlendirme ölçütlerini içeren sözlük:
            - 'accuracy': Doğruluk skoru
            - 'f1': F1 skoru
            - 'precision': Kesinlik değeri
            - 'recall': Duyarlılık değeri
    """
    # Tuple'dan tahmin ve etiketleri çıkar
    predictions, labels = eval_pred
    
    # Olasılık değerlerini (logits) en yüksek olasılığa sahip sınıf indeksine dönüştür
    predictions = np.argmax(predictions, axis=1)
    
    # İkili sınıflandırma için ölçütleri hesapla
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    
    # Doğruluk skorunu hesapla
    acc = accuracy_score(labels, predictions)
    
    # Tüm ölçütleri bir sözlük içinde döndür
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Model Eğitim işlemleri

## Model yükleme

In [ ]:
model_adi = "dbmdz/bert-base-turkish-128k-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_adi)
tokenizer = AutoTokenizer.from_pretrained(model_adi)
model.to(cihaz)

In [ ]:
# Eğitim ve test veri kümelerini hazırla
egitim_veri_kumesi = verileri_hazirla(egitim_veri, tokenizer)
dogrulama_veri_kumesi = verileri_hazirla(dogrulama_veri, tokenizer)
sinama_veri_kumesi = verileri_hazirla(sinama_veri, tokenizer)

## Eğitim argümanlarını ve eğiticiyi ayarla

In [ ]:
# Eğitim argümanlarını tanımla
training_args = TrainingArguments(
    output_dir=model_cikti_dizini,  # Eğitilen modelin kaydedileceği dizin
    num_train_epochs=1,  # Toplam eğitim dönemi (epoch) sayısı
    report_to=["tensorboard"],  # Eğitim metriklerinin raporlanacağı araç
    per_device_train_batch_size=8,  # Her GPU/CPU için eğitim batch boyutu
    per_device_eval_batch_size=1,  # Her GPU/CPU için değerlendirme batch boyutu
    gradient_accumulation_steps=16,  # Gradyanların biriktirileceği adım sayısı (efektif batch boyutunu artırır)
    warmup_steps=0,  # Isınma adımı sayısı (learning rate'in kademeli olarak artması için)
    weight_decay=0.01,  # Ağırlıkların L2 düzenlileştirme (regularization) katsayısı
    logging_dir=model_log_dizini,  # Logların kaydedileceği dizin
    logging_steps=5,  # Her kaç adımda bir log kaydedileceği
    eval_strategy="steps",  # Değerlendirme stratejisi ("epoch" veya "steps")
    eval_steps=5,  # Her kaç adımda bir değerlendirme yapılacağı
    save_strategy="steps",  # Model kaydetme stratejisi ("epoch" veya "steps")
    save_steps=5,  # Her kaç adımda bir modelin kaydedileceği
    load_best_model_at_end=True,  # Eğitim sonunda en iyi modelin yüklenmesi
    metric_for_best_model="f1",  # En iyi modeli seçmek için kullanılacak metrik
    # Konsola çıktı için ek parametreler
    logging_strategy="steps",  # Log kaydetme stratejisi
    logging_first_step=True,  # İlk adımın da loglanması için
)

# Eğiticiyi (Trainer) oluştur
trainer = Trainer(
    model=model,  # Eğitilecek model
    args=training_args,  # Eğitim argümanları
    train_dataset=egitim_veri_kumesi,  # Eğitim veri kümesi
    eval_dataset=dogrulama_veri_kumesi,  # Değerlendirme için kullanılacak doğrulama veri kümesi
    compute_metrics=compute_metrics,  # Metrikleri hesaplayan fonksiyon
)

## Eğitim ve sınama işlemleri

In [ ]:
# Modeli eğit
trainer.train()

In [ ]:
train_results = trainer.state.log_history
with open(train_sonuc_json_path, "w") as f:
    json.dump(train_results, f, indent=4)

In [ ]:
# Değerlendirme
eval_results = trainer.evaluate(sinama_veri_kumesi)
print(f"Değerlendirme sonuçları: {eval_results}")

with open(eval_sonuc_json_path, "w") as f:
    json.dump(eval_results, f, indent=4)
print(f"Değerlendirme sonuçları {eval_sonuc_json_path} dosyasına kaydedildi.")

## Eğitilen modeli kaydetme

In [ ]:
# Eğitilen modeli kaydet
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Görselleştirme

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Json okuma fonksiyonu

In [ ]:
def json_dosya_oku(dosya_yolu: str) -> dict:
    """
    JSON dosyasını okur ve içeriğini Python nesnesine dönüştürerek döndürür.
    
    Bu fonksiyon, belirtilen dosya yolundaki JSON formatındaki dosyayı açar,
    içeriğini okur ve JSON verilerini Python veri yapısına (sözlük, liste vb.) 
    dönüştürerek döndürür. Dosya UTF-8 kodlaması ile açılır.
    
    Args:
        dosya_yolu (str): Okunacak JSON dosyasının tam yolu
        
    Returns:
        dict: JSON dosyasının içeriğinden oluşturulan Python nesnesi (genellikle sözlük)
        
    Raises:
        FileNotFoundError: Belirtilen dosya bulunamazsa
        json.JSONDecodeError: Dosya geçerli bir JSON formatında değilse
    """
    # Dosyayı UTF-8 kodlamasıyla okuma modunda aç
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        # JSON içeriğini Python nesnesine dönüştür ve döndür
        return json.load(f)

## Çizim tarzını ayarla

In [ ]:
# Matplotlib'in görünüm stilini ggplot olarak ayarla
# Bu, R dilindeki ggplot2 paketine benzer bir görsel stil sağlar
plt.style.use('ggplot')

# Seaborn kütüphanesinin font ölçeğini 1.2 olarak ayarla
# Bu, grafikteki tüm yazıların boyutunu artırır ve daha okunaklı hale getirir
sns.set(font_scale=1.2)

# Matplotlib'te çizilen grafiklerin varsayılan boyutunu 14x8 inç olarak ayarla
# Bu, daha büyük ve detaylı grafikler oluşturulmasını sağlar
plt.rcParams['figure.figsize'] = (14, 8)

# Grafik başlıklarının yazı boyutunu 16 punto olarak ayarla
# Başlıkların daha belirgin olmasını sağlar
plt.rcParams['axes.titlesize'] = 16

# Grafiklerde kullanılacak yazı tipini DejaVu Sans olarak ayarla
# Bu yazı tipi, Türkçe karakterleri de doğru şekilde gösterir
plt.rcParams['font.family'] = 'DejaVu Sans'

## Görselleştirme fonksiyonları

In [ ]:
def egitim_sureci_gorsellestirilmesi(train_results: list, cikti_dizini: str = model_log_dizini) -> None:
    """
    Eğitim sürecindeki ölçütleri görselleştirir ve grafikleri kaydeder.
    
    Bu fonksiyon, model eğitimi sırasında elde edilen sonuçları alır ve
    çeşitli grafikler oluşturarak eğitim sürecinin görsel analizini sağlar.
    
    Args:
        train_results (list): Eğitim sürecinde elde edilen sonuçları içeren liste
        cikti_dizini (str): Grafiklerin kaydedileceği dizin (varsayılan: model_log_dizini)
    
    Returns:
        None: Fonksiyon grafikleri belirtilen dizine kaydeder
    """
    # Eğitim ve değerlendirme adımlarını ayıkla
    train_steps = [x for x in train_results if 'loss' in x and 'eval_loss' not in x]
    eval_steps = [x for x in train_results if 'eval_loss' in x and 'epoch' in x]
    
    # Grafikler için gerekli verileri hazırla
    epochs_train = [step.get('epoch', 0) for step in train_steps]  # Eğitim adımlarına ait döngü değerleri
    loss_train = [step.get('loss', 0) for step in train_steps]     # Eğitim kaybı değerleri
    
    epochs_eval = [step.get('epoch', 0) for step in eval_steps]    # Değerlendirme adımlarına ait döngü değerleri
    loss_eval = [step.get('eval_loss', 0) for step in eval_steps]  # Doğrulama kaybı değerleri
    accuracy_eval = [step.get('eval_accuracy', 0) for step in eval_steps]  # Doğruluk değerleri
    f1_eval = [step.get('eval_f1', 0) for step in eval_steps]              # F1 skoru değerleri
    precision_eval = [step.get('eval_precision', 0) for step in eval_steps] # Kesinlik değerleri
    recall_eval = [step.get('eval_recall', 0) for step in eval_steps]       # Duyarlılık değerleri
    
    # Kayıp (Loss) Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_train, loss_train, 'b-', marker='o', label='Eğitim Kaybı')
    plt.plot(epochs_eval, loss_eval, 'r-', marker='s', label='Doğrulama Kaybı')
    plt.xlabel('Döngü')
    plt.ylabel('Kayıp Değeri')
    plt.title('Eğitim ve Doğrulama Kaybının Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'kayip_grafigi.png'), dpi=300, bbox_inches='tight')
    
    # Doğruluk, F1, Kesinlik ve Duyarlılık Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_eval, accuracy_eval, 'b-', marker='o', label='Doğruluk')
    plt.plot(epochs_eval, f1_eval, 'g-', marker='s', label='F1 Skoru')
    plt.plot(epochs_eval, precision_eval, 'r-', marker='^', label='Kesinlik')
    plt.plot(epochs_eval, recall_eval, 'm-', marker='D', label='Duyarlılık')
    plt.xlabel('Döngü')
    plt.ylabel('Ölçüt Değeri')
    plt.title('Doğrulama ölçütlerinin Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'ölçütler_grafigi.png'), dpi=300, bbox_inches='tight')
    
    # Öğrenme Oranı Grafiği (eğer varsa)
    if 'learning_rate' in train_steps[0]:
        learning_rates = [step.get('learning_rate', 0) for step in train_steps]
        plt.figure(figsize=(14, 6))
        plt.plot(epochs_train, learning_rates, 'g-', marker='o')
        plt.xlabel('Döngü')
        plt.ylabel('Öğrenme Oranı')
        plt.title('Öğrenme Oranının Döngü Üzerinden Değişimi', fontweight='bold')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'ogrenme_orani_grafigi.png'), dpi=300, bbox_inches='tight')
    plt.close('all')  # Tüm açık şekilleri kapat


def degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc: dict, cikti_dizini: str = model_log_dizini) -> None:
    """
    Değerlendirme sonuçlarını görselleştirir ve grafikleri kaydeder.
    
    Bu fonksiyon, model değerlendirme sonuçlarını alır ve çeşitli grafikler oluşturarak
    modelin performansını görsel olarak analiz etmeyi sağlar.
    
    Args:
        eval_sonuc (dict): Model değerlendirme sonuçlarını içeren sözlük
        cikti_dizini (str): Grafiklerin kaydedileceği dizin (varsayılan: model_log_dizini)
    
    Returns:
        None: Fonksiyon grafikleri belirtilen dizine kaydeder
    """
    # Değerlendirme ölçütlerini çıkar
    olcutler = ['accuracy', 'f1', 'precision', 'recall']
    olcut_isimleri = ['Doğruluk', 'F1 Skoru', 'Kesinlik', 'Duyarlılık']
    
    olcut_degerleri = []
    for olcut in olcutler:
        key = f"eval_{olcut}"
        if key in eval_sonuc:
            olcut_degerleri.append(eval_sonuc[key])  # Ölçüt değerini listeye ekle
        else:
            olcut_degerleri.append(0)  # Eksik ölçüt değeri için 0 ekle
    
    # Ölçüt değerlerini çubuk grafik olarak göster
    plt.figure(figsize=(12, 8))
    bars = plt.bar(olcut_isimleri, olcut_degerleri, color=['#3274A1', '#E1812C', '#3A923A', '#C03D3E'])
    plt.ylim(0, 1.1)  # Y ekseni sınırları (ölçüt değerleri genellikle 0-1 arasında)
    plt.title('Model Değerlendirme Sonuçları', fontweight='bold')
    plt.ylabel('Değer')
    
    # Çubukların üzerine değerleri ekleme
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.4f}', ha='center', va='bottom', fontsize=12)
    
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'degerlendirme_sonuclari.png'), dpi=300, bbox_inches='tight')
    
    # Kayıp ve çalışma süresi grafiği (eğer bu değerler varsa)
    if 'eval_loss' in eval_sonuc and 'eval_runtime' in eval_sonuc:
        plt.figure(figsize=(10, 6))
        plt.bar(['Kayıp', 'Çalışma Süresi (sn)'], 
                [eval_sonuc['eval_loss'], eval_sonuc['eval_runtime']], 
                color=['#C03D3E', '#3274A1'])
        plt.title('Değerlendirme Kaybı ve Çalışma Süresi', fontweight='bold')
        plt.grid(axis='y')
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'kayip_ve_sure.png'), dpi=300, bbox_inches='tight')
    plt.close('all')  # Tüm açık şekilleri kapat


def egitim_ozet_tablosu(train_results: list, eval_sonuc: dict, cikti_dizini: str = model_log_dizini) -> None:
    """
    Eğitim sürecinin özet tablosunu oluşturur ve kaydeder.
    
    Bu fonksiyon, eğitim ve değerlendirme sonuçlarını alarak önemli metrikleri
    içeren bir özet tablo oluşturur ve kaydeder.
    
    Args:
        train_results (list): Eğitim sürecinde elde edilen sonuçları içeren liste
        eval_sonuc (dict): Model değerlendirme sonuçlarını içeren sözlük
        cikti_dizini (str): Tablonun kaydedileceği dizin (varsayılan: model_log_dizini)
    
    Returns:
        None: Fonksiyon özet tabloyu belirtilen dizine kaydeder
    """
    # Son eğitim ölçütlerini bul
    son_egitim = None
    for step in reversed(train_results):  # Sondan başlayarak ara
        if 'train_runtime' in step:  # Eğitim çalışma süresi varsa bu son eğitim adımıdır
            son_egitim = step
            break
    
    if son_egitim:
        # Tablo verilerini hazırla
        data = {
            'Ölçüt': ['Eğitim Süresi (sn)', 'Ör./sn', 'Adım/sn', 'Toplam FLOPS', 
                      'Son Kayıp', 'Son Döngü', 'Doğruluk', 'F1 Skoru'],
            'Değer': [
                son_egitim.get('train_runtime', 'N/A'),  # Eğitim süresi
                son_egitim.get('train_samples_per_second', 'N/A'),  # Saniyedeki örnek sayısı
                son_egitim.get('train_steps_per_second', 'N/A'),  # Saniyedeki adım sayısı
                f"{son_egitim.get('total_flos', 0) / 1e12:.2f} T" if 'total_flos' in son_egitim else 'N/A',  # FLOPS değeri
                son_egitim.get('train_loss', 'N/A'),  # Son eğitim kaybı
                son_egitim.get('epoch', 'N/A'),  # Son döngü
                eval_sonuc.get('eval_accuracy', 'N/A'),  # Doğruluk değeri
                eval_sonuc.get('eval_f1', 'N/A')  # F1 skoru
            ]
        }
        
        # DataFrame oluştur
        df = pd.DataFrame(data)
        
        # Tabloyu görselleştir
        plt.figure(figsize=(10, 6))
        table = plt.table(cellText=df.values, colLabels=df.columns, 
                         loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 1.5)  # Tablo hücrelerinin boyutunu ayarla
        plt.axis('off')  # Eksenleri gizle
        plt.title('Eğitim Özet Tablosu', fontweight='bold', y=0.9)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'egitim_ozeti.png'), dpi=300, bbox_inches='tight')
        plt.close('all')  # Tüm açık şekilleri kapat

## Sonuçları oku ve görselleştir

In [ ]:
train_results = json_dosya_oku(train_sonuc_json_path)
eval_sonuc = json_dosya_oku(eval_sonuc_json_path)

In [ ]:
# Görselleştirmeleri oluştur
print("Eğitim süreci görselleştiriliyor...")
egitim_sureci_gorsellestirilmesi(train_results)

print("Değerlendirme sonuçları görselleştiriliyor...")
degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc)

print("Eğitim özet tablosu oluşturuluyor...")
egitim_ozet_tablosu(train_results, eval_sonuc)